In [102]:
#Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [58]:
#scraping the news
news_url='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response=requests.get(news_url)
soup=BeautifulSoup(response.text,'html.parser')
news_title=soup.find_all('div', class_='rollover_description_inner')[0].text
news_title=news_title.split('\n')
news_title=news_title[1]

news_p=soup.find_all('div', class_='content_title')[0].text
news_p=news_p.split('\n')
news_p=news_p[2]
news_p

"The Launch Is Approaching for NASA's Next Mars Rover, Perseverance"

In [116]:
#scraping featured image
browser= webdriver.Chrome('chromedriver.exe')
browser.get('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
button=browser.find_element_by_link_text("FULL IMAGE")
button.click()
pictures=browser.find_elements_by_tag_name('img')
time.sleep(3)
picture=browser.find_element_by_class_name("fancybox-image")
featured_image_url=picture.get_attribute('src')

In [117]:
#get tweet
browser= webdriver.Chrome('chromedriver.exe')
browser.get('https://twitter.com/marswxreport?lang=en')
tweet=browser.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div/div/div/div[2]/section/div/div/div/div[1]/div/div/article/div/div/div/div[2]/div[2]/div[2]')
mars_weather=tweet.text

In [120]:
#get pandas table
tables = pd.read_html('https://space-facts.com/mars/')
df=tables[0]
df.columns=['Record', 'Measurement']
df.set_index('Record', inplace=True)
html_table=df.to_html()

In [121]:
#get images of hemispheres
browser.get('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
links=browser.find_elements_by_class_name('itemLink')

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measurement</th>\n    </tr>\n    <tr>\n      <th>Record</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [124]:
browser= webdriver.Chrome('chromedriver.exe')
browser.get('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
links=browser.find_elements_by_class_name('itemLink')

In [151]:
url_list=[]
for link in links:
    title=link.text
    url=(link.get_attribute('href'))
    #print(title)
    url_list.append(url)

In [164]:
url_list1=[]
for item in url_list:
    if item not in url_list2:
        url_list2.append(item)
    else:
        continue

In [173]:
url

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
